In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

import time
from tqdm import tqdm

In [2]:
import openpyxl
from openpyxl.styles import PatternFill,Color
from openpyxl import Workbook
from random import *

## Edge에 옵션을 주는 방법, 그리고 Headless
- pip install msedge-selenium-tools selenium==3.141

In [3]:
from msedge.selenium_tools import Edge, EdgeOptions

In [4]:
options = EdgeOptions()
options.use_chromium = True 
options.add_argument("headless")
options.add_argument("disable-gpu")

# edge 드라이버를 직접 다운로드 받을 때의 방법
driver = Edge('./msedgedriver.exe', options=options)

True

In [32]:
# 크롤링할 단어
# space는 +로 바꿈
def get_keyword(text):
    return text.replace(" ","%20")

In [33]:
# 정렬 우선순위
def sort_kind(index):
    if index == 1:
        return 'vcount' # 추천순
    elif index == 2:
        return 'date' # 최신순
    else:
        return 'none' # 정확도순

In [34]:
keyword = '여친 선물'
driver.get('https://kin.naver.com/search/list.nhn?query=' + get_keyword(keyword) )

# Headless 제대로 동작하나요? => 스크린샷
driver.get_screenshot_as_file('./tmp/naver_main.png')


# random.uniform(a,b): a~b까지 동일 확률로 하나의 수
time.sleep(uniform(0.1, 1.0))

- 최근 6개월 간 동향

In [35]:
page_index = 1
start = datetime.now() + timedelta(days=-180)
start =datetime.strftime(start,'%Y.%m.%d')
print(start, type(start))

end = datetime.strftime(datetime.now(),'%Y.%m.%d')
print(end, type(end))

period = '&period=' + start + '.%7C' + end + '.'
print(period)

2020.06.12 <class 'str'>
2020.12.09 <class 'str'>
&period=2020.06.12.%7C2020.12.09.


In [36]:
today = end
today

'2020.12.09'

In [37]:
# url 저장
f = open('./result/url_list_' + get_keyword(keyword) + '_' + today + '.txt', 'w')
f

<_io.TextIOWrapper name='./result/url_list_여친%20선물_2020.12.09.txt' mode='w' encoding='cp949'>

In [38]:
page_url = []
while True:
    time.sleep(uniform(0.1, 1.0))
    driver.get('https://kin.naver.com/search/list.nhn?' + "&sort=" + sort_kind(0) + '&query=' + get_keyword(keyword) + period + "&section=kin" + "&page=" + str(page_index))
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    tags = soup.select('ul.basic1 li > dl > dt > a')
    for tag in tags:
        url = tag.get('href')
        page_url.append(url)
        f.write(url + '\n')
    
    post_number = soup.select_one('.number').text.lstrip('(').rstrip(')').replace(',','')
    current_number = int(post_number.split('/')[0].split('-')[1])
    total_number = int(post_number.split('/')[1])

    print('현재글수:',current_number)   
    if current_number == total_number:
        break
    else:
        page_index += 1

print('총:',total_number)
f.close()

현재글수: 10
현재글수: 20
현재글수: 30
현재글수: 40
현재글수: 47
총: 47


- 엑셀로 저장할 때 파일 이름

In [39]:
excelname = './result/' + get_keyword(keyword) + '_' + today + '_crawling_result.xlsx'
excelname

'./result/여친%20선물_2020.12.09_crawling_result.xlsx'

- url 파일 한 줄씩 읽어 리스트로 만들기

In [40]:
with open('./result/url_list_' + get_keyword(keyword) + '_' + today + '.txt', 'r') as f:
    list_file = f.readlines()
list_file = [line.rstrip('\n') for line in list_file] 

In [41]:
print(type(list_file))
list_file[0]

<class 'list'>


'https://kin.naver.com/qna/detail.nhn?d1id=5&dirId=511&docId=372320059&qb=7Jes7LmcIOyEoOusvA==&enc=utf8&section=kin&rank=1&search_sort=0&spq=0'

In [42]:
wb = Workbook()
sheet = wb.active
sheet.append(['제목','질문','답변'])

In [43]:
for j in range(1, 4):
    sheet.cell(row = 1, column=j).fill = PatternFill(start_color='808080', end_color='808080', fill_type='solid')

for url in tqdm(list_file):
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'html.parser')
    # main 제목
    title = soup.select_one('div.title').text.lstrip('"').rstrip('"').strip()

    try:
        content_text = soup.select_one('div.c-heading__content').text.strip()
    except:
        content_text = ""
    
    answer_lists = soup.select("div._endContentsText.c-heading-answer__content-user")

    for i, answer in enumerate(answer_lists):
        lines = answer.select('div')
        answer_text = ""
        for line in lines:
            answer_text += line.text.strip()
        
        if i == 0:
            sheet.append([title,content_text,answer_text])
        else:
            sheet.append(["","",answer_text])
    wb.save(excelname)

100%|██████████| 47/47 [00:36<00:00,  1.30it/s]
